In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [9]:
df = pd.read_csv("ndtv_data_final.csv")

In [20]:
# Cek Missing Values ---
print("Jumlah Missing Value per Kolom:")
print(df.isnull().sum())

# Tangani missing values
# Jika kolom numerik -> isi dengan median
# Jika kolom kategorikal -> isi dengan modus
for col in df.columns:
    if df[col].isnull().sum() > 0:
        if df[col].dtype in ['int64', 'float64']:
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)

print("\nTotal missing value setelah imputasi:", df.isnull().sum().sum())

Jumlah Missing Value per Kolom:
Unnamed: 0                0
Name                      0
Brand                     0
Model                     0
Battery capacity (mAh)    0
Screen size (inches)      0
Touchscreen               0
Resolution x              0
Resolution y              0
Processor                 0
RAM (MB)                  0
Internal storage (GB)     0
Rear camera               0
Front camera              0
Operating system          0
Wi-Fi                     0
Bluetooth                 0
GPS                       0
Number of SIMs            0
3G                        0
4G/ LTE                   0
Price                     0
PPI                       0
dtype: int64

Total missing value setelah imputasi: 0


Data kosong bisa muncul karena kesalahan pencatatan atau data yang memang tidak tersedia. Kalau ada nilai kosong di kolom angka, kita isi dengan median (nilai tengah) supaya tidak terlalu terpengaruh oleh nilai yang terlalu besar atau kecil. Kalau yang kosong ada di kolom teks seperti nama brand atau sistem operasi, kita isi dengan nilai yang paling sering muncul (modus). Dengan
begitu, tidak ada lagi data kosong yang bisa bikin error waktu model dijalankan.

In [21]:
# Cek dan hapus duplikat ---
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]
print(f"\nJumlah data duplikat yang dihapus: {before - after}")


Jumlah data duplikat yang dihapus: 0


Kadang ada data yang sama persis muncul lebih dari satu kali. Data seperti ini tidak memberi informasi tambahan, malah bisa bikin model jadi bias karena “belajar” dari data yang berulang. Kita hapus data ganda ini supaya setiap baris benar-benar unik.


In [22]:
# Tangani Outlier ---
# Metode IQR
numeric_cols = df.select_dtypes(include="number").columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower, upper)

print("\nOutlier sudah ditangani dengan metode IQR (data di-clip pada batas bawah & atas).")


Outlier sudah ditangani dengan metode IQR (data di-clip pada batas bawah & atas).


Outlier itu nilai yang jauh banget dari nilai lainnya, misalnya harga yang terlalu tinggi atau RAM yang tidak masuk akal.
Kita pakai metode IQR (Interquartile Range) untuk membatasi nilai-nilai ini.
Kalau ada nilai yang terlalu tinggi atau terlalu rendah dari batas normal, kita potong ke batas yang wajar supaya tidak mengganggu model.

In [23]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = df.copy()

df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])
print("\nData numerik setelah standarisasi:")
print(df_scaled[numeric_cols].head())



Data numerik setelah standarisasi:
     Unnamed: 0      Name     Brand     Model  Battery capacity (mAh)  \
279   -2.664890  0.808844  1.365824  0.982504                1.414138   
335   -2.446787  0.367656 -0.044181 -0.003566                1.988481   
371   -2.306578 -1.527817 -1.051328 -0.726684                0.101354   
374   -2.294894  1.299053  1.466538 -0.036435                1.988481   
402   -2.185843  1.364414  1.567253  1.722056               -0.607550   

     Screen size (inches)  Touchscreen  Resolution x  Resolution y  Processor  \
279                   0.0          0.0      0.744334       0.73291        0.0   
335                   0.0          0.0      0.744334       0.73291        0.0   
371                   0.0          0.0      0.744334       0.73291        0.0   
374                   0.0          0.0      0.744334       0.73291        0.0   
402                   0.0          0.0      0.744334       0.73291        0.0   

     ...  Front camera  Operating syst

Setelah data bersih, langkah selanjutnya adalah menyesuaikan skala antar kolom angka.
Misalnya, harga ponsel bisa jutaan, tapi ukuran layar cuma beberapa inci.
Kalau langsung dimasukkan ke model, nilai yang besar (seperti harga) bisa mendominasi dan membuat model jadi tidak seimbang.

Untuk itu, kita gunakan StandardScaler, yang akan membuat setiap kolom angka memiliki rata-rata 0 dan standar deviasi 1.
Dengan cara ini, semua fitur punya pengaruh yang seimbang saat model belajar.

In [24]:
from sklearn.preprocessing import LabelEncoder

cat_cols = df_scaled.select_dtypes(include=["object"]).columns
le = LabelEncoder()

for col in cat_cols:
    df_scaled[col] = le.fit_transform(df_scaled[col])

print("\nContoh hasil encoding:")
print(df_scaled[cat_cols].head())



Contoh hasil encoding:
Empty DataFrame
Columns: []
Index: [279, 335, 371, 374, 402]


Kolom seperti “Brand”, “Model”, atau “Operating System” berisi teks, sedangkan model machine learning hanya bisa memproses angka.
Supaya bisa digunakan, kita ubah semua kolom teks menjadi angka menggunakan LabelEncoder.

Contohnya, kalau di kolom “Brand” ada “Samsung”, “Apple”, dan “Xiaomi”, maka masing-masing akan diubah menjadi 0, 1, dan 2.
Tujuannya supaya model bisa membedakan kategori satu dengan yang lain, meskipun aslinya berupa teks.

In [25]:
# Contoh fitur baru: PPI (Pixels Per Inch)
df_scaled["PPI"] = np.sqrt(df_scaled["Resolution x"]**2 + df_scaled["Resolution y"]**2) / df_scaled["Screen size (inches)"]

print("\nFitur baru 'PPI' berhasil ditambahkan:")
print(df_scaled[["Resolution x", "Resolution y", "Screen size (inches)", "PPI"]].head())



Fitur baru 'PPI' berhasil ditambahkan:
     Resolution x  Resolution y  Screen size (inches)  PPI
279      0.744334       0.73291                   0.0  inf
335      0.744334       0.73291                   0.0  inf
371      0.744334       0.73291                   0.0  inf
374      0.744334       0.73291                   0.0  inf
402      0.744334       0.73291                   0.0  inf


Tahap ini bertujuan untuk menambah fitur baru agar model bisa mengenali pola dengan lebih baik.
Kita buat satu fitur baru bernama PPI (Pixel Per Inch), yaitu ukuran kerapatan layar ponsel.

In [26]:
from sklearn.model_selection import train_test_split

X = df_scaled.drop("Price", axis=1)
y = df_scaled["Price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\nData berhasil dibagi menjadi:")
print(f"Training set: {X_train.shape[0]} record")
print(f"Testing set: {X_test.shape[0]} record")



Data berhasil dibagi menjadi:
Training set: 169 record
Testing set: 43 record


Setelah semua data sudah siap, langkah terakhir adalah membagi data menjadi dua bagian:

Data training (80%): digunakan untuk melatih model.

Data testing (20%): digunakan untuk menguji seberapa bagus model yang sudah dibuat.

Pembagian ini dilakukan dengan fungsi train_test_split dari library sklearn.
Tujuannya agar kita bisa mengetahui apakah model hanya “menghafal” data latih atau benar-benar bisa menebak data baru dengan baik.